In [ ]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import joblib
import os
from xgboost import XGBRegressor

# Initialize the main window
root = tk.Tk()
root.title("Prediction of IWQI")

# Create the main canvas
canvas = tk.Canvas(root, width=600, height=450, background='#FFFFFF')
canvas.pack()

# Function to create labels on the canvas
def create_label(text, font, fg, bg, x, y):
    label = tk.Label(root, text=text, font=font, fg=fg, bg=bg)
    canvas.create_window(x, y, anchor="w", window=label)
    return label

# Check if the XGBoost model file exists
model_path_xgb = os.path.expanduser("~/Desktop/XGB1.joblib")
if not os.path.exists(model_path_xgb):
    messagebox.showerror("Error", "XGBoost model file not found.")
    raise SystemExit  # Exit if the model file is not found

# Load the XGBoost model
model_xgb = joblib.load(model_path_xgb)

# Define the GUI elements
label_inputdefinetitle2 = create_label('Prediction of Irrigation Water Quality Index', ('Palatino Linotype', 18, 'bold', 'underline'), '#0000FF', '#FFFF00', 35,20)
label_inputdefinetitle = create_label('Definition of Parameters', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', '#FFFFFF', 10,65)
param_labels = [
    'X1: Total Dissoloved Solids (TDS)',
    'X2: Potential of Hydrogen (pH)',
    'X3: Electrical Conductivity (EC)',
    'X4: Bicarbonate (HCO3)',
    'X5: Cholride (Cl)',
    'X6: Sulfate (SO4)',
    'X7: Sodium (Na)',
    'X8: Calcium (Ca)',
    'X9: Magnesium (Mg)',
    'X10: Potassium (K)',
]
for i, text in enumerate(param_labels, start=1):
    create_label(text, ('Georgia', 14), '#006600', '#FFFFFF', 10, 65 + i*30)

label_inputs = create_label('Inputs', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', '#FFFFFF', 450, 65)

entry_fields = ['X1', 'X2', 'X3', 'X4','X5', 'X6', 'X7', 'X8', 'X9', 'X10']
entries = {}
for i, field in enumerate(entry_fields, start=1):
    label = create_label(f'{field}: ', ('Georgia', 14), '#FF0000', '#FFFFFF', 380, 70 + i*30)
    entry = tk.Entry(root)
    canvas.create_window(500, 70 + i*30, window=entry)
    entries[field] = entry

label_output = create_label('Output:', ('Palatino Linotype', 16, 'bold'), '#000000', '#FFFFFF', 20, 420)
label_result = create_label('           IWQI', ('Georgia', 16), '#FF0000', '#FFFFFF', 100, 420)

# Predict button command function
def predict():
    input_values = []
    all_inputs_collected = True

    # Collect input data
    for field in entry_fields:
        value = entries[field].get()
        try:
            # Convert input to float and append if valid
            input_values.append(float(value))
        except ValueError:
            # Show error and mark input as incomplete
            messagebox.showerror("Error", f"Invalid input for {field}. Please enter a positive numeric value.")
            all_inputs_collected = False
            break

    # Proceed only if all inputs are valid
    if all_inputs_collected:
        if len(input_values) == 10:
            # Proceed with prediction using the collected input_values
            try:
                prediction_result = model_xgb.predict(np.array([input_values]))[0]
                label_result.config(text=       f'IWQI= {prediction_result:.2f}')
            except Exception as e:
                messagebox.showerror("Prediction Error", str(e))
        else:
            messagebox.showerror("Error", "All 10 inputs are required for the prediction.")

# Predict button
predict_button = tk.Button(root, text='Calculate', font=('Georgia', 12), command=predict, fg='#FFFFFF', bg='green')
canvas.create_window(450, 420, window=predict_button)

# Run the main loop
root.mainloop()